<a href="https://colab.research.google.com/github/jamirpradojunior/JPJ/blob/main/20220328_Ciclos_de_Mercado_sazonalidade_anual.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://tradingcomdados.com/images/logotipo/logotipo-trading-com-dados.svg" width="300">

---
# **Ciclos de Mercado: avaliando a sazonalidade anual do IBOV**
---

## **Contexto:**

Você provavelmente já ouviu a expressão "*Sell in May and go away!*", que em uma tradução literal seria algo como "*Venda em maio e vá embora!*". O fato é que além de uma rima legal, essa expressão traz um contexto de sazonalidade de mercado muito interessante e que, sem dúvida, merece nossa atenção.<p>
Estudos mostram que a expressão original é ainda mais curiosa: "*Sell in May, go away, and come back on St. Leger’s Day*" = Venda em maio, vá embora e volte no dia de 'Saint Leger' (15 de setembro). A estratégia envolve o lucro a partir na sazonalindade anual do mercado. Basicamente, você venderia todas as suas ações em maio, quando o mercado historicamente apresenta um desempenho abaixo do esperado. No mesmo ano, entre outubro e novembro, você os compra novamente, quando o mercado de ações começa a subir novamente.<p>
Em um artigo intitulado "*The Halloween Indicator, 'Sell in May and Go Away': Another Puzzle*", publicado na American Economic Review (2002), os autores Jacobsen and Bouman avaliam e documentam esse padrão para 36 de 37 mercados avaliados (incluindo países desenvolvidos e emergentes). Eles reportam que esse "indicador Halloween" é particularmente forte na Europa, e surpreendentemente pode ser observado desde 1694 no mercado do UK:<p>
>[...] we find this inherited wisdom to be true in 36 of the 37 developed and emerging markets studied in our sample. The Sell in May effect tends to be particularly strong in European countries and is robust over time. Sample evidence, for instance, shows that in the UK the effect has been noticeable since 1694.

Por fim, embora tenham testado diversos preditores que pudessem explicar esse padrão, não chegaram a nenhuma conclusão sobre o que poderia de fato explicar esse "quebra-cabeça":
>[...] While we have examined a number of possible explanations, none of these appears to convincingly explain the puzzle.

Nós, da Trading com Dados, apresentamos aqui um código exemplificando o padrão no IBOV e S&P 500. E em seus ativos preferidos, quer testar?

# 1. Bibliotecas utilizadas

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# 2. Entendendo um código HTML

## 2.1. Código HTML bruto *vs* Aparência HTML

HTML é basicamente uma linguagem de marcação, onde cada **tag** (ex., \<html>...<\\html>) representa algo dentro da hierarquia da página. Neste caso, **\<html>** indica o início de um conteúdo html e **<\\html>** indica o fim deste conteúdo. Dentro desta tag temos outras tags, cada uma referente a determinada estrutura do HTML. Veja abaixo um código HTML bruto e logo abaixo como está sua "aparência"

    ------------------------------------------------------------------------------------------
    <!DOCTYPE html>
    <html>
        <head>
            <title>Page Title</title>
            <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
        </head>

        <body>
    
            <div>
                <h1>Isto é um título</h1>
                <p>Isto é um parágrafo</p>
                <a href='https://www.google.com.br'>Link do google</a>
            </div>

            <div>
                <h2>Lista não ordenada</h2>

                <ul>
                    <li>Café</li>
                    <li>Chá</li>
                    <li>Leite</li>
                </ul>  

                <h2>Lista ordenada</h2>

                <ol>
                    <li>Café</li>
                    <li>Chá</li>
                    <li>Leite</li>
                </ol>  
            </div>

            <div>
                <h2>Exemplo de uma tabela</h2>

                <table>
                    <tr>
                        <th>Nome</th>
                        <th>Sobrenome</th>
                        <th>Idade</th>
                    </tr>
                    <tr>
                        <td>Jill</td>
                        <td>Smith</td>
                        <td>50</td>
                    </tr>
                    <tr>
                        <td>Eve</td>
                        <td>Jackson</td>
                        <td>94</td>
                    </tr>
                </table> 
            </div>
        
        </body>
    </html>
    ------------------------------------------------------------------------------------------

Para visualizar o HTML estruturado deverá exportar o código como HTML
Alguns sites permitem essa visualização. Ex: https://pt.infobyip.com/htmlandjavascripteditor.php

Principais **tags** e suas descrições simplificadas
* h1: Indica o título da página ou da sessão em que ela está contida. Podemos também usar h2, h3... para indicar subtítulos.
* div: Serve para organizar e dividir partes do código
* span: Serve para organizar os elementos de uma mesma linha
* p: Indica um parágrafo
* a: Indica um link, onde o link vem representado por 'href' dentro da tag
* ul: Indica listas não ordenadas
* ol: Indica listas ordenadas
* li: Indica os elementos da lista 'ul' ou 'ol'
* table: Indica uma tabela
* tr: Indica o começo de uma linha em 'table'
* th: Indica o head em 'table'
* td: Indica uma linha em 'table'


## 2.2. Inspecionando o código desenvolvido em cada site

Por mais que os desenvolvedores web em HTML tentem seguir regras de estruturação na criação das páginas, é impossível que todas estejam no mesmo padrão. Por exemplo, uma notícia no **InfoMoney** pode ser formatada em um padrão diferente de uma notícia no **Bloomberg** ou no **Investing**. Por isso, devemos entender como cada página está estruturada antes de começar a extrair qualquer informação dela. Nessas etapas recomendamos o uso do Google Chrome, devido à sua facilidade em mostrar de forma intuitiva o código fonte no ***Ambiente de desenvolvedor - DevTools***

Para navegar no Ambiente de Desenvolvedor entre em um site e aperte F12. Depois selecione o botão 'Inspect' (canto esquerdo superior, Ctrl+Shift+C)<p>
https://www.infomoney.com.br/

Após explorar o botão Inspect no código HTML do site, clique com o botão direito e vá em "Salvar como..." > "Salvar" (Nome: exemplo_html | Tipo: Página da web completa)

In [ ]:
# Para carregarmos o html salvo, primeiro vamos verificar em qual diretório a IDE está trabalhando (varia entre elas, por ex., Jupyter e Colab)

import os
os.getcwd()

'C:\\Users\\jamir\\Documents\\GitHub\\desenvolvimento_interno'

In [ ]:
# Comando para abrir o html. O output será gigante pois contem todo o código da página
# A variável criada "html_fonte" é uma string. Para evitar sobrecarregar o output, vamos imprimir apenas alguns caracteres
# Basta retirar o indexador caso queira visualizar todo o código
# Por curiosidade, vamos imprimir antes de tudo o número de caracteres do html importado

html_fonte = open("C:\\Users\\jamir\\Desktop\\exemplo_html.html", "r", encoding='utf8').read()
print("O número de caracteres do código html importado é",len(html_fonte))
html_fonte[0:5000]

O número de caracteres do código html importado é 438454


'<!DOCTYPE html>\n<!-- saved from url=(0029)https://www.infomoney.com.br/ -->\n<html lang="pt-BR" class="jetpack-lazy-images-js-enabled hydrated"><head><meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n    <style data-styles="">{visibility:hidden}.hydrated{visibility:inherit}</style><style data-styles="">{visibility:hidden}.hydrated{visibility:inherit}</style><style data-styles="">terms-policy,terms-policy-checkbox,terms-policy-modal{visibility:hidden}.hydrated{visibility:inherit}</style><style data-styles="">cookies-policy,cookies-policy-disclaimer,cookies-policy-list,cookies-policy-modal-footer,cookies-policy-modal-tabs,cookies-policy-modal{visibility:hidden}.hydrated{visibility:inherit}</style><style data-styles="">soma-link,soma-time-picker,soma-token,soma-datepicker,soma-popover,soma-accordion-item,soma-menu,soma-quantity,soma-alert,soma-avatar,soma-button-link,soma-card-header,soma-chip,soma-dialog-warning,soma-icon-button,soma-input-bank-password,soma-modal,so

Da forma como está carregado (como uma grande string) é praticamente impossível trabalhar com o código (muito diferente de quando inspecionamos no Google Chrome através do DevTools. Nesse ponto, o uso da biblioteca ***bs4 (BeautifulSoup)*** vem auxiliar na organização e abstração de toda a string criada a partir do código HTML, de forma a facilitar o uso no python

In [ ]:
# Organizar o html_fonte html.parser"</b> como parâmetros da função <b>BeautifulSoup</b>, conseguimos organizar nosso código HTML:</p>

html = BeautifulSoup(html_fonte, "html.parser")

html

NameError: name 'BeautifulSoup' is not defined

In [ ]:
<p>Percebemos que o output é bem mais amigável dessa vez, mas o mais importante está nos métodos de filtro e busca que podemos fazer com ele. Apesar do <b>'bs4'</b> apresentar muitos métodos úteis, inicialmente podemos começar com apenas 3 deles:</p>

# 3. Caso 1 - Webscraping com Rendimentos de Fundos Imobiliários (FIIs)

Funds Explorer - muito legal para obtenção de dados de FIIs <p>
Dados de mais @ FIIs, distribuídos em 9 setores e 24 colunas de atributos <p>
https://www.fundsexplorer.com.br/ranking

In [ ]:
# Importamos nossa url objeto de webscraping usando a biblioteca requests

FII_url = requests.get("https://www.fundsexplorer.com.br/ranking")
FII_url

<Response [200]>

In [ ]:
# Transformar o código em texto
# Output reduzido

FII_url.text[0:1000]

'<!DOCTYPE html>\n<html lang="pt-br">\n\n<head>\n<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"7ae9c6eb2d","applicationID":"1576126089","transactionName":"IAkLQhFbWVUBQhdKVwcDFl8EWhpLBV5TUVwESQxYB1FN","queueTime":3,"applicationTime":15,"agent":""}</script>\n<script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"7ae9c6eb2d",applicationID:"1576126089"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n,r){return function(){return s.recordSupportability("API/"+e+"/called"),o(t+e,[u.now()].concat(c(a

In [ ]:
# Analisar o arquivo com a BeutifulSoup (HTML parsing)

FII_bs4 = BeautifulSoup(FII_url.text, "html.parser")
FII_bs4.head()

[<script type="text/javascript">window.NREUM||(NREUM={});NREUM.info={"beacon":"bam.nr-data.net","errorBeacon":"bam.nr-data.net","licenseKey":"7ae9c6eb2d","applicationID":"1576126089","transactionName":"IAkLQhFbWVUBQhdKVwcDFl8EWhpLBV5TUVwESQxYB1FN","queueTime":3,"applicationTime":15,"agent":""}</script>,
 <script type="text/javascript">(window.NREUM||(NREUM={})).init={ajax:{deny_list:["bam.nr-data.net"]}};(window.NREUM||(NREUM={})).loader_config={licenseKey:"7ae9c6eb2d",applicationID:"1576126089"};window.NREUM||(NREUM={}),__nr_require=function(t,e,n){function r(n){if(!e[n]){var i=e[n]={exports:{}};t[n][0].call(i.exports,function(e){var i=t[n][1][e];return r(i||e)},i,i.exports)}return e[n].exports}if("function"==typeof __nr_require)return __nr_require;for(var i=0;i<n.length;i++)r(n[i]);return r}({1:[function(t,e,n){function r(){}function i(t,e,n,r){return function(){return s.recordSupportability("API/"+e+"/called"),o(t+e,[u.now()].concat(c(arguments)),n?null:this,r),n?void 0:this}}var o=

In [ ]:
# Extração da tabela. Antes devemos inspecionar o Ambiente de Desenvolvedor no próprio site (F12)

FII_table = FII_bs4.findAll(attrs = {'id': 'scroll-wrapper'})
table = FII_table[0].findAll('table')

In [ ]:
# Output gigante, mas vemos que é um objeto Result Set de bs4

type(table)

bs4.element.ResultSet

In [ ]:
# Transformar o <table>...</table> em um Pandas DataFrame
# Vamos substituir o separador de casas decimais (na página estão como vírgula) por '.' 

FII_df = pd.read_html(str(FII_table[0]))[0]

In [ ]:
FII_df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,...,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,"R$ 2,31",15952.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 65.746.443,59","R$ 6,98",33.0,NaN,NaN,NaN,NaN,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,"R$ 14,70",44041.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 432.739.853,66","R$ 10,89",135.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,Lajes Corporativas,"R$ 6,23",93.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 27.391.929,40","R$ 13,94",45.0,NaN,NaN,NaN,NaN,"0,00%","100,00%",1
3,ALMI11,Lajes Corporativas,"R$ 945,00",39.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 250.487.663,18","R$ 2.253,05",42.0,NaN,NaN,NaN,NaN,"64,05%",NaN,1
4,PABY11,Híbrido,"R$ 8,34",1.0,"R$ 0,00","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ -8.707.885,16","R$ -11,48",-73.0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,VJFD11,Híbrido,NaN,NaN,"R$ 0,86","0,00%","0,00%","0,00%","0,00%","0,00%",...,"R$ 650.064.545,92","R$ 133,32",NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
273,WTSP11B,Híbrido,"R$ 40,10",12.0,"R$ 0,48","1,01%","2,53%","4,46%","6,33%","0,84%",...,"R$ 77.799.824,27","R$ 78,38",51.0,"0,40%","3,55%","3,97%","3,97%",NaN,NaN,0
274,BTRA11,Outros,"R$ 99,46",8311.0,"R$ 0,90","0,92%","2,81%","4,99%","0,00%","0,94%",...,"R$ 347.617.328,85","R$ 103,32",96.0,NaN,NaN,NaN,NaN,NaN,NaN,0
275,EDGA11,Lajes Corporativas,"R$ 19,66",7731.0,"R$ 0,11","0,49%","1,29%","2,71%","6,04%","0,43%",...,"R$ 252.515.454,02","R$ 66,24",30.0,"0,16%","0,00%","0,16%","0,16%","48,81%",NaN,1


In [ ]:
FII_df.columns

Index(['Códigodo fundo', 'Setor', 'Preço Atual', 'Liquidez Diária',
       'Dividendo', 'DividendYield', 'DY (3M)Acumulado', 'DY (6M)Acumulado',
       'DY (12M)Acumulado', 'DY (3M)Média', 'DY (6M)Média', 'DY (12M)Média',
       'DY Ano', 'Variação Preço', 'Rentab.Período', 'Rentab.Acumulada',
       'PatrimônioLíq.', 'VPA', 'P/VPA', 'DYPatrimonial',
       'VariaçãoPatrimonial', 'Rentab. Patr.no Período',
       'Rentab. Patr.Acumulada', 'VacânciaFísica', 'VacânciaFinanceira',
       'QuantidadeAtivos'],
      dtype='object')

In [ ]:
# Note que temos problemas para reconhecer os valores de algumas colunas (ex., preço atual) como números

FII_df['Preço Atual']

0        R$ 2,31
1       R$ 14,70
2        R$ 6,23
3      R$ 945,00
4        R$ 8,34
         ...    
272          NaN
273     R$ 40,10
274     R$ 99,46
275     R$ 19,66
276     R$ 28,45
Name: Preço Atual, Length: 277, dtype: object

In [ ]:
# Vamos substituir o "R$ " por "vazio"

FII_df['Preço Atual'] = FII_df['Preço Atual'].replace(to_replace=r'^R\$ ', value='',regex=True)

In [ ]:
# Outro problema é o separador decimal (atualmente como vírgula). Entretanto, não podemos simplesmente substituir por '.' diretamente pois podem existir valores com '.' como separador de milhar
# Nessa caso, melhor fazer em duas etapas

FII_df['Preço Atual']

0        2,31
1       14,70
2        6,23
3      945,00
4        8,34
        ...  
272       NaN
273     40,10
274     99,46
275     19,66
276     28,45
Name: Preço Atual, Length: 277, dtype: object

In [ ]:
# Primeira etapa: substituir qq '.' por 'vazio'

FII_df['Preço Atual'] = FII_df['Preço Atual'].str.replace('.','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# Segunda etapa: substituir as ',' por '.' (separador decimal padrão para executarmos alguma análise de dados)
# Agora é possível converter os dados em float

FII_df['Preço Atual'] = FII_df['Preço Atual'].str.replace(',','.').astype(float)
FII_df['Preço Atual']

0        2.31
1       14.70
2        6.23
3      945.00
4        8.34
        ...  
272       NaN
273     40.10
274     99.46
275     19.66
276     28.45
Name: Preço Atual, Length: 277, dtype: float64

In [ ]:
# Análises simples como valor máximo são possíveis agora nesta coluna

max(FII_df['Preço Atual'])

76000.0

In [ ]:
# Agora faremos o mesmo para a coluna "DY (12M)Acumulado" (dividend yield acumulado nos últimos 12 meses)

FII_df['DY (12M)Acumulado']

0      0,00%
1      0,00%
2      0,00%
3      0,00%
4      0,00%
       ...  
272    0,00%
273    6,33%
274    0,00%
275    6,04%
276    5,12%
Name: DY (12M)Acumulado, Length: 277, dtype: object

In [ ]:
# Substituir '%' por 'vazio' e ',' por '.'

FII_df['DY (12M)Acumulado'] = FII_df['DY (12M)Acumulado'].str.replace('%','')
FII_df['DY (12M)Acumulado'] = FII_df['DY (12M)Acumulado'].str.replace(',','.').astype(float)
max(FII_df['DY (12M)Acumulado'])

21.8

In [ ]:
# Remover os NAs antes de análises exploratórias

FII_df.dropna(subset = ['Preço Atual','DY (12M)Acumulado','Setor'], inplace=True)
FII_df

,Códigodo fundo,Setor,Preço Atual,Liquidez Diária,Dividendo,DividendYield,DY (3M)Acumulado,DY (6M)Acumulado,DY (12M)Acumulado,DY (3M)Média,...,PatrimônioLíq.,VPA,P/VPA,DYPatrimonial,VariaçãoPatrimonial,Rentab. Patr.no Período,Rentab. Patr.Acumulada,VacânciaFísica,VacânciaFinanceira,QuantidadeAtivos
0,FIVN11,Shoppings,2.31,15952.0,"R$ 0,00","0,00%","0,00%","0,00%",0.00,"0,00%",...,"R$ 65.746.443,59","R$ 6,98",33.0,NaN,NaN,NaN,NaN,"56,00%",NaN,1
1,BZLI11,Títulos e Val. Mob.,14.70,44041.0,"R$ 0,00","0,00%","0,00%","0,00%",0.00,"0,00%",...,"R$ 432.739.853,66","R$ 10,89",135.0,NaN,NaN,NaN,NaN,NaN,NaN,0
2,XTED11,Lajes Corporativas,6.23,93.0,"R$ 0,00","0,00%","0,00%","0,00%",0.00,"0,00%",...,"R$ 27.391.929,40","R$ 13,94",45.0,NaN,NaN,NaN,NaN,"0,00%","100,00%",1
3,ALMI11,Lajes Corporativas,945.00,39.0,"R$ 0,00","0,00%","0,00%","0,00%",0.00,"0,00%",...,"R$ 250.487.663,18","R$ 2.253,05",42.0,NaN,NaN,NaN,NaN,"64,05%",NaN,1
4,PABY11,Híbrido,8.34,1.0,"R$ 0,00","0,00%","0,00%","0,00%",0.00,"0,00%",...,"R$ -8.707.885,16","R$ -11,48",-73.0,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271,CNES11,Lajes Corporativas,34.67,365.0,"R$ 0,13","0,39%","1,19%","2,14%",3.39,"0,40%",...,"R$ 255.388.232,33","R$ 86,59",40.0,"0,15%","0,00%","0,15%","0,15%","56,70%",NaN,1
273,WTSP11B,Híbrido,40.10,12.0,"R$ 0,48","1,01%","2,53%","4,46%",6.33,"0,84%",...,"R$ 77.799.824,27","R$ 78,38",51.0,"0,40%","3,55%","3,97%","3,97%",NaN,NaN,0
274,BTRA11,Outros,99.46,8311.0,"R$ 0,90","0,92%","2,81%","4,99%",0.00,"0,94%",...,"R$ 347.617.328,85","R$ 103,32",96.0,NaN,NaN,NaN,NaN,NaN,NaN,0
275,EDGA11,Lajes Corporativas,19.66,7731.0,"R$ 0,11","0,49%","1,29%","2,71%",6.04,"0,43%",...,"R$ 252.515.454,02","R$ 66,24",30.0,"0,16%","0,00%","0,16%","0,16%","48,81%",NaN,1


In [ ]:
import plotly.express as px

fig = px.scatter(FII_df, x="Preço Atual", y="DY (12M)Acumulado",color='Setor')
fig.show()

In [ ]:
import numpy as np

FII_df['log Preço Atual'] = np.log(FII_df["Preço Atual"]+1)

fig = px.scatter(FII_df, x="log Preço Atual", y="DY (12M)Acumulado",color='Setor')
fig.show()

In [ ]:
# Nesta análise, os FIIs dos setores "Outros" e "Hospitais" e "Títulos e Val. Mob." parecem ser melhores em relação ao retorno de dividendos

pivot_table_FIIs = pd.pivot_table(FII_df, values="DY (12M)Acumulado", index=["Setor"], columns=[],aggfunc=('mean','median'))
pivot_table_FIIs.sort_values('mean',ascending=False)

,mean,median
Setor,,
Outros,8.690882,9.32
Hospital,8.436667,9.33
Títulos e Val. Mob.,7.741047,9.47
Logística,6.930909,7.63
Lajes Corporativas,6.590000,6.60
Híbrido,6.562791,7.63
Hotel,4.920000,5.41
Shoppings,4.685000,5.82
Residencial,2.285714,0.00


In [ ]:
# Visualização gráfica por boxplots

fig = px.box(FII_df, x="Setor", y="DY (12M)Acumulado")
fig.show()

# 4. Caso 2 - Webscraping em Promoção diárias no Mercado Livre
